# NOAA Severe Weather Events EDA

Data download site. May be able to automate with selenium.
[Storm Events Bulk Data Download](https://www.ncdc.noaa.gov/stormevents/ftp.jsp)

Possible Scraping Targets
[Billion Dollar Weather Events](https://www.ncdc.noaa.gov/billions/overview)

[Wikipedia Costliest Atlantic Hurricanes](https://en.wikipedia.org/wiki/List_of_costliest_Atlantic_hurricanes)

I've downloaded one year's worth of Severe Weather Events data. From here, I'll investigate how to isolate tropical storms, and then merge them with the data from the atlantic tropical storms database

In [1]:
# Necessary Imports
import pandas as pd
import numpy as np
import datetime as dt

In [2]:
data = pd.read_csv('StormEvents_details-ftp_v1.0_d2018_c20180918.csv')

In [4]:
data.columns

Index(['BEGIN_YEARMONTH', 'BEGIN_DAY', 'BEGIN_TIME', 'END_YEARMONTH',
       'END_DAY', 'END_TIME', 'EPISODE_ID', 'EVENT_ID', 'STATE', 'STATE_FIPS',
       'YEAR', 'MONTH_NAME', 'EVENT_TYPE', 'CZ_TYPE', 'CZ_FIPS', 'CZ_NAME',
       'WFO', 'BEGIN_DATE_TIME', 'CZ_TIMEZONE', 'END_DATE_TIME',
       'INJURIES_DIRECT', 'INJURIES_INDIRECT', 'DEATHS_DIRECT',
       'DEATHS_INDIRECT', 'DAMAGE_PROPERTY', 'DAMAGE_CROPS', 'SOURCE',
       'MAGNITUDE', 'MAGNITUDE_TYPE', 'FLOOD_CAUSE', 'CATEGORY', 'TOR_F_SCALE',
       'TOR_LENGTH', 'TOR_WIDTH', 'TOR_OTHER_WFO', 'TOR_OTHER_CZ_STATE',
       'TOR_OTHER_CZ_FIPS', 'TOR_OTHER_CZ_NAME', 'BEGIN_RANGE',
       'BEGIN_AZIMUTH', 'BEGIN_LOCATION', 'END_RANGE', 'END_AZIMUTH',
       'END_LOCATION', 'BEGIN_LAT', 'BEGIN_LON', 'END_LAT', 'END_LON',
       'EPISODE_NARRATIVE', 'EVENT_NARRATIVE', 'DATA_SOURCE'],
      dtype='object')

In [5]:
# Search for subtropical storm alberto on may 25
data.head()

,BEGIN_YEARMONTH,BEGIN_DAY,BEGIN_TIME,END_YEARMONTH,END_DAY,END_TIME,EPISODE_ID,EVENT_ID,STATE,STATE_FIPS,...,END_RANGE,END_AZIMUTH,END_LOCATION,BEGIN_LAT,BEGIN_LON,END_LAT,END_LON,EPISODE_NARRATIVE,EVENT_NARRATIVE,DATA_SOURCE
0,201806,6,1810,201806,6,1810,125578,753161,NEBRASKA,31,...,36.0,N,OSHKOSH,41.9300,-102.2100,41.9300,-102.2100,Severe storms developed in the Nebraska Panhan...,Hail predominately penny size with some quarte...,CSV
1,201806,6,1741,201806,6,1741,125578,753160,NEBRASKA,31,...,1.0,NW,BINGHAM,42.0300,-102.1000,42.0300,-102.1000,Severe storms developed in the Nebraska Panhan...,Hail mainly quarter size with some half dollar...,CSV
2,201806,30,2330,201806,30,2332,125988,755273,VERMONT,50,...,3.0,WSW,WEST BERKSHIRE,44.9565,-72.8699,44.9565,-72.8699,Vermont and northern NY influenced by heat rid...,Numerous trees downed by thunderstorm winds.,CSV
3,201806,30,2345,201806,30,2345,125988,755929,VERMONT,50,...,0.0,NE,BELVIDERE JCT,44.7316,-72.7474,44.7316,-72.7474,Vermont and northern NY influenced by heat rid...,At least half dozen trees downed or snapped al...,CSV
4,201806,6,1824,201806,6,1824,125578,753163,NEBRASKA,31,...,5.0,NW,GRANT,40.9000,-101.7900,40.9000,-101.7900,Severe storms developed in the Nebraska Panhan...,Tornado briefly touched down in a field 5 mile...,CSV


In [9]:
X = data[(data.BEGIN_YEARMONTH == 201805)&(data.BEGIN_DAY == 25)&(data.STATE == 'FLORIDA')].EPISODE_NARRATIVE
for thing in X:
    print(thing)

A 31 year old male drowned, likely due to rip currents, near the Tropic Winds Condos in the 17000 block of Front Beach Road in the Panama City Beach area.  He was face down in the water when friends located him.  Single red flags were flying.||In addition, a 28 year old male died near the 700 block of Scenic Gulf Drive along the Walton county coast.  The man was swimming in the evening after lifeguards had left the beach.  Red flags were flying.
A 31 year old male drowned, likely due to rip currents, near the Tropic Winds Condos in the 17000 block of Front Beach Road in the Panama City Beach area.  He was face down in the water when friends located him.  Single red flags were flying.||In addition, a 28 year old male died near the 700 block of Scenic Gulf Drive along the Walton county coast.  The man was swimming in the evening after lifeguards had left the beach.  Red flags were flying.
Afternoon showers and storms developed along the sea breezes along each coast. The Gulf coast sea br